# 🦢 AI Stress Testing Engine (Black Swan Generator) v3

## 목표
Neural Path Integral(NPI)을 활용하여 **"금융 위기 시나리오(Black Swan)"를 효율적으로 생성**합니다.

## v3 수정사항 (Aggressive Mode)
- **Control Bound 확대**: `1.0` -> `5.0` (Neural Engine 수정 필요)
- **Targeted Loss**: 전체 평균 하락이 아닌, **"Crash Threshold 돌파"**에 집중하는 Hinge Loss 사용
- **Weak Regularization**: 제약 조건을 완화하여 과감한 움직임 허용

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

import sys
sys.path.append('..')

import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from src.physics_engine import MarketSimulator
from src.neural_engine import DriftNet

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
# Monkey Patching DriftNet to increase Control Bound
# 소스코드를 직접 수정하지 않고, 런타임에 클래스 메서드를 교체하여 Bound를 5.0으로 늘립니다.

def aggressive_forward(self, S, v, t, avg_S=None):
    # ... (Pre-processing code duplicated for context)
    S_norm = torch.log(S + 1e-8)
    v_norm = torch.log(v + 1e-8)
    t_norm = t * torch.ones_like(S)
    
    if avg_S is None:
        avg_S = S
    avg_S_norm = torch.log(avg_S + 1e-8)
    
    x = torch.stack([S_norm, v_norm, t_norm, avg_S_norm], dim=-1)
    raw_output = self.net(x).squeeze(-1)
    
    # BOUND INCREASED TO 5.0 for Aggressive Crash Generation
    return 5.0 * torch.tanh(raw_output)

# Apply patch
DriftNet.forward = aggressive_forward
print("DriftNet patched: Control Bound increased to 5.0 🚀")

## 1. Setup & Baseline

In [ ]:
# Market Parameters
S0 = 100.0
v0 = 0.04
r = 0.05
T = 0.25
dt = 0.001

# Heston Parameters
kappa = 2.0; theta = 0.04; xi = 0.5; rho = -0.7

# Settings
num_paths = 50000
CRASH_THRESHOLD = 0.80  # 20% Drop
K_crash = CRASH_THRESHOLD * S0

simulator = MarketSimulator(mu=r, kappa=kappa, theta=theta, xi=xi, rho=rho, device=device)

# Standard MC (from previous runs, we know it's ~3.24%)
crash_prob_mc = 0.0324 
print(f"Baseline MC Crash Prob: {crash_prob_mc*100:.2f}%")

## 2. Neural Crash Generator v3 (Aggressive)

In [ ]:
# Initialize
drift_net = DriftNet(hidden_dim=64).to(device)
optimizer = torch.optim.Adam(drift_net.parameters(), lr=1e-3)

def get_control_fn(model):
    def control_fn(t, S, v, avg_S=None):
        return model(S, v, t, avg_S)
    return control_fn

control_fn = get_control_fn(drift_net)

In [ ]:
# Training Loop v3: Targeted Hinge Loss
num_iters = 500
batch_size = 10000

loss_history = []
crash_rate_history = []

print("Training v3 (Aggressive Mode)...")
for i in range(num_iters):
    optimizer.zero_grad()
    
    S, v, log_weights, _, _ = simulator.simulate_controlled(
        S0=S0, v0=v0, T=T, dt=dt, num_paths=batch_size,
        model_type='heston', control_fn=control_fn
    )
    
    S_T = S[:, -1]
    
    # === V3 LOSS: Targeted Hinge Loss ===
    # We want S_T < K_crash.
    # Penalty = ReLU(S_T - (K_crash - margin))
    # 즉, Crash Threshold보다 높으면 페널티 부여. 낮으면 OK.
    
    margin = 2.0  # 안전마진: 78.0 이하로 내려가도록 유도
    target_level = K_crash - margin
    
    # Hinge Loss: target_level보다 높은 만큼 Loss
    dist_penalty = F.relu(S_T - target_level)
    
    # Mean of penalties
    loss_dist = dist_penalty.mean()
    
    # Regularization (Weaker than v2)
    # 0.1 -> 0.001 (Allow stronger controls)
    loss_reg = 0.001 * (log_weights ** 2).mean()
    
    loss = loss_dist + loss_reg
    
    loss.backward()
    torch.nn.utils.clip_grad_norm_(drift_net.parameters(), 1.0)
    optimizer.step()
    
    crash_rate = (S_T < K_crash).float().mean().item()
    loss_history.append(loss.item())
    crash_rate_history.append(crash_rate)
    
    if i % 100 == 0:
        print(f"Iter {i:3d}: Loss={loss.item():.4f}, Crash Rate={crash_rate*100:.2f}%")

print("Training Complete!")

In [ ]:
# Plot Training Progress
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(loss_history)
plt.title('Loss (Hinge + Weak Reg)')

plt.subplot(1, 2, 2)
plt.plot(crash_rate_history, color='red')
plt.axhline(y=crash_prob_mc, color='gray', linestyle='--')
plt.title('Crash Rate')
plt.show()

In [ ]:
# Generate & Evaluate
drift_net.eval()
with torch.no_grad():
    S_ai, v_ai, log_weights_ai, _, _ = simulator.simulate_controlled(
        S0=S0, v0=v0, T=T, dt=dt, num_paths=num_paths,
        model_type='heston', control_fn=control_fn
    )

crash_mask_ai = S_ai[:, -1] < K_crash
num_crashes_ai = crash_mask_ai.sum().item()
crash_rate_ai = num_crashes_ai / num_paths

# Efficiency Boost Calculation
boost = crash_rate_ai / max(crash_prob_mc, 1e-6)

print(f"="*60)
print(f"       AI STRESS TESTING ENGINE v3 (Aggressive)")
print(f"="*60)
print(f"Crash Rate: {crash_rate_ai*100:.2f}% (MC Baseline: {crash_prob_mc*100:.2f}%)")
print(f"⚡ Efficiency Boost: {boost:.1f}x")
print(f"Generated Crashes: {num_crashes_ai} / {num_paths}")

# Visualization
plt.figure(figsize=(10, 5))
plt.hist(S_ai[:, -1].cpu().numpy(), bins=100, color='red', alpha=0.7, label='AI Distribution')
plt.axvline(x=K_crash, color='black', linestyle='--', linewidth=2, label='Crash Threshold')
plt.xlabel('Terminal Stock Price')
plt.title('Terminal Price Distribution (v3 Aggressive)')
plt.legend()
plt.show()